In [ ]:
from copy import deepcopy
import math
import sys

import matplotlib.pylab as plt
import numpy as np

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

sys.path.append('../crazyflie-audio/python/')

# A. Basics

In [ ]:
from generate_results import generate_signals_analytical, generate_signals_pyroom

def plot_spectrum(spectrum, polar):
    if polar: 
        plt.polar(BeamFormer.theta_scan, spectrum[0, :])
        for i, gt_angle_deg in enumerate(gt_angles_deg):
            plt.axvline(gt_angle_deg / 180 * np.pi, color=f"C{i + 1}")
    else:
        plt.semilogy(BeamFormer.theta_scan_deg, spectrum[0, :])
        for i, gt_angle_deg in enumerate(gt_angles_deg):
            plt.axvline(gt_angle_deg, color=f"C{i + 1}")

def get_buffers(sanity_check=False):
    indices = np.where(frequencies == frequency_hz)[0]
    
    buffer_f_list = []
    if sanity_check:
        buffer_list = []
        
    #### moving mic array ####
    for mics, time in zip(mics_list, times_list):
        buffers = np.zeros((mics.shape[0], n_buffer))
        
        # superposition
        for phase_offset, source in zip(phase_offsets, sources):
            if simulation_type == "pyroom":
                signals_received =  generate_signals_pyroom(
                    source, mics, frequency_hz, time, noise=signal_noise,
                    phase_offset=phase_offset) 
            elif simulation_type == "analytical":
                signals_received =  generate_signals_analytical(
                    source, mics, frequency_hz, time, noise=signal_noise, 
                    phase_offset=phase_offset) 
            else:
                raise ValueError(simulation_type)
            buffers_source = signals_received[:, time_index:time_index + n_buffer]
            buffers += buffers_source
            
        if sanity_check:
            buffer_list.append(buffers)

        buffer_f = np.fft.rfft(buffers)[:, indices]
        buffer_f_list.append(buffer_f.T)

    
    #### multi-mic array ####
    mics_array = np.concatenate([*mics_list])
    buffer_multimic = np.zeros((mics_array.shape[0], n_buffer))
    
    # superposition
    for phase_offset, source in zip(phase_offsets, sources):
        if simulation_type == "pyroom":
            signals_multimic = generate_signals_pyroom(source, mics_array, frequency_hz, time=0, noise=signal_noise, phase_offset=phase_offset)
        elif simulation_type == "analytical":
            signals_multimic = generate_signals_analytical(source, mics_array, frequency_hz, time=0, noise=signal_noise, phase_offset=phase_offset)
        else:
            raise ValueError(simulation_type)
        buffer_multimic += signals_multimic[:, time_index:time_index + n_buffer]
    buffer_f_multimic = np.fft.rfft(buffer_multimic)[:, indices]
    
    #### sanity check delay correction ####
    if sanity_check:
        counter = 0
        delta_t =  times_list[1] - times_list[0]
        print(f'period of signal {period:.2e}')
        print(f'delta_t % period: {delta_t % period :.2e}')

        n_plots = len(buffer_list) * mics_drone.shape[0]

        times = np.arange(n_plot) / FS
        fig, axs = plt.subplots(n_plots, 2, sharex=True)
        fig.set_size_inches(10, 10)
        for i in range(n_plots): # n_mics
            label=f"mic{i}"
            axs[i, 0].plot(times, buffer_multimic[i, :n_plot], label=label, color=f"C{i}")
            axs[i, 0].set_title(label)
        for j, (degree, buffer, time) in enumerate(zip(degrees, buffer_list, times_list_noisy)):
            for i in range(buffer.shape[0]): # n_mics
                label=f"mic{counter}"
                phase_shift_deg = from_0_to_2pi(-2 * np.pi * time * frequency_hz) * 180 / np.pi
                time_shift = period * phase_shift_deg / 360
                axs[counter, 1].axvline(time_shift)
                axs[counter, 1].plot(times, buffer[i, :n_plot], color=f"C{counter}")
                axs[counter, 1].set_title(label)
                counter += 1
            
    return buffer_f_list, buffer_f_multimic

In [ ]:
from generate_results import *
from mic_array import get_square_array, get_uniform_array
from audio_stack.beam_former import rotate_mics

simulation_type = 'pyroom'
#simulation_type = 'anayltical'

method = 'das'
#method = 'das'

plot_covariance = False

frequency_hz = 4134.375 # Hz
baseline = 0.108  # meters, square side of mic array
gt_distance = 10  # meters, distance of source
gt_angles_deg = [50] # angle of ground truth
phase_offsets = [0.0]
angular_velocity_deg = 20 # deg/sec, velocity of drone

max_distance = SPEED_OF_SOUND / frequency_hz / 2
mics_drone = get_square_array(baseline=baseline, delta=0) # 4 x 2
#mics_drone = get_uniform_array(2, baseline=baseline) # 4 x 2
mics_drone -= np.mean(mics_drone, axis=0) # center the drone

# sanity check
#times_list = np.array([0, 0.8, 1.5]) #3., 6.]) #1, 1.5]) # in seconds

# uniform array, works very well
times_list = np.array([30/angular_velocity_deg, 
                       60/angular_velocity_deg, 
                       90/angular_velocity_deg]) # in seconds

# very small (too small) movements
#times_list = np.array([0, 0.2, 0.3, 0.5, 0.4]) #3., 6.]) #1, 1.5]) # in seconds

In [ ]:
### parameters
n_buffer = 2048
time_index = 2000 # idx where signal is non-zero for all positions, found heuristically
period = 1/frequency_hz

# make sure times are not an exact multiple of period
#times_list[1:] += period/2
# make sure sampling times coincide with times
times_list = np.array([round(t * FS) / FS for t in times_list])

signal_noise = 1e-3  # noise added to signals
mics_noise = 0 #1e-3  # noise added to mic positions (rigid)
degree_noise = 0 #5 # noise added to each movement, in degrees
time_quantization = 6 # number of decimal places to keep
time_noise = 0 #1e-6 # noise added to recording times

frequencies = np.fft.rfftfreq(n_buffer, 1/FS)
if frequency_hz not in frequencies:
    raise ValueError(f'frequency_hz not in available frequency bins: {list(frequencies)}')

In [ ]:
np.random.seed(1)

degrees = times_list * angular_velocity_deg
print('degrees:', degrees)
print('recording times in seconds:', times_list)

assert DURATION > max(times_list)

sources = []
for gt_angle_deg in gt_angles_deg:
    gt_angle_rad = gt_angle_deg / 180 * np.pi
    sources.append(gt_distance * np.array([np.cos(gt_angle_rad), np.sin(gt_angle_rad)]))

# create noisy versions
mics_drone_noisy = deepcopy(mics_drone)
times_list_noisy = deepcopy(times_list)
degrees_noisy = deepcopy(degrees)
if mics_noise > 0:
    mics_drone_noisy += np.random.normal(scale=mics_noise, size=mics_drone.shape)
if time_noise > 0:
    times_list_noisy += np.random.normal(scale=time_noise, size=times_list.shape)
if time_quantization > 0:
    times_list_noisy = np.round(times_list_noisy, time_quantization)
if degree_noise > 0:
    degrees_noisy += np.random.normal(scale=degree_noise, size=degrees.shape)

# we think we move to "degrees" position but we actually move to degrees_noisy.
mics_list = [rotate_mics(mics_drone_noisy, orientation_deg=degree) for degree in degrees_noisy]

plt.figure()
for i, (degree, mics) in enumerate(zip(degrees, mics_list)):
    plt.scatter(*mics.T, label=f'real {degree:.0f} deg', color=f"C{i}")
        
mics_clean = [rotate_mics(mics_drone, orientation_deg=degree) for degree in degrees]
for i, (degree, mics) in enumerate(zip(degrees, mics_clean)):
    plt.scatter(*mics.T, label=f'theo {degree:.0f} deg', color=f"C{i}", marker="x")
plt.axis('equal')
plt.legend(bbox_to_anchor=[1, 1], loc='upper left')
pass

In [ ]:
# sanity checks
for mics, time in zip(mics_list, times_list):
    fig, ax = plt.subplots()
    
    buffers = np.zeros((mics.shape[0], n_buffer))
    for gt_angle_deg in gt_angles_deg:
        gt_angle_rad = gt_angle_deg * np.pi / 180.0
        source = gt_distance * np.array([np.cos(gt_angle_rad), np.sin(gt_angle_rad)])
        signals_received_an =  generate_signals_analytical(
            source, mics, frequency_hz, time, noise=signal_noise, ax=ax) 
        signals_received_py =  generate_signals_pyroom(
            source, mics, frequency_hz, time, noise=signal_noise, ax=ax) 
        
        if simulation_type == "pyroom":
            buffers += signals_received_py[:, time_index:time_index + n_buffer]
        elif simulation_type == "analytical":
            buffers += signals_received_an[:, time_index:time_index + n_buffer]
        else:
            raise ValueError(simulation_type)
            
    ax.set_xlim(1300, 1400) 
    ax.legend()
    ax.set_title(f'signals at time {time:.3f}')
    
    fig, axs = plt.subplots(buffers.shape[0])
    n_plot = 50
    fig.suptitle(f'first {n_plot} samples of received buffers')
    for i, ax in enumerate(axs):
        ax.plot(buffers[i, :n_plot], color=f'C{i}')

In [ ]:
#buffer_f_list, buffer_f_multimic = get_buffers(sanity_check=True)
buffer_f_list, buffer_f_multimic = get_buffers()

# 3. Do beamforming, separately

In [ ]:
from audio_stack.beam_former import BeamFormer

def separate_doa(plot=False, polar=True):
    for buffer_f, mics in zip(buffer_f_list, mics_list) :
        beam_former = BeamFormer(mic_positions=mics)
        R = beam_former.get_correlation(buffer_f)
        
        if method == 'das':
            spectrum = beam_former.get_das_spectrum(R, np.array([frequency_hz]))
        else:
            spectrum = beam_former.get_mvdr_spectrum(R, np.array([frequency_hz]), 
                                                     inverse='pinv')

        if plot:
            
            if plot_covariance:
                plt.figure()
                plt.matshow(np.angle(R[0]))
                plt.colorbar()
                
            plt.figure()
            plot_spectrum(spectrum, polar)
            
separate_doa(plot=True)

# 3.2 Multi-mic DOA

In [ ]:
def multi_mic_doa(plot=False, polar=True):
    mics_array = np.concatenate([*mics_list])
    beam_former = BeamFormer(mics_array)
    R_multimic = beam_former.get_correlation(buffer_f_multimic.T)
    frequencies = np.array([frequency_hz])
    
    if method == 'das':
        spectrum_multimic = beam_former.get_das_spectrum(R_multimic, frequencies)
    else:
        spectrum_multimic = beam_former.get_mvdr_spectrum(R_multimic, frequencies)

    if plot:
        if plot_covariance:
            plt.figure()
            plt.matshow(np.angle(R_multimic[0]))
            
        plot_spectrum(spectrum_multimic, polar)
        plt.title('multimic')

multi_mic_doa(plot=True)

# 3.3 Kinetic DOA

In [ ]:
def kinetic_doa(plot=False, polar=True):
    frequencies = np.array([frequency_hz])

    beam_former = BeamFormer(mic_positions=mics_drone)
    beam_former.init_multi_estimate(frequencies, combination_n=len(degrees))
    beam_former.init_dynamic_estimate(frequencies, combination_n=len(degrees), 
                                      combination_method='sum', normalization_method='none')

    if plot:
        plt.figure()
        
    for i, (sig_f, time) in enumerate(zip(buffer_f_list, times_list_noisy)):
        beam_former.add_to_multi_estimate(sig_f, frequencies, time, degrees[i])
        spec = beam_former.add_signals_to_dynamic_estimates(sig_f, frequencies, degrees[i], method=method)
        
        if plot:
            plot_spectrum(spec, polar)
            
    spectrum_dynamic = beam_former.get_dynamic_estimate()
    spectrum_delayed = beam_former.get_multi_estimate(method=method)

    if plot:
        plt.title('shifted spectra')
        
        plt.figure()
        plot_spectrum(spectrum_dynamic, polar)
        plt.title('spectrum_dynamic')
        
        plt.figure()
        plot_spectrum(spectrum_delayed, polar)
        plt.title('spectrum_delayed')
        
kinetic_doa(plot=True)

# B. Multiple source test

In [ ]:
polar = True
method = "das"

gt_angles_deg = [50, 90]#, 180, 270]
phase_offsets = np.random.uniform(0, 2*np.pi, size=len(gt_angles_deg))
sources = []
simulation_type = "analytical"

plt.figure()
plt.scatter(*mics_drone.T)
for gt_angle_deg in gt_angles_deg:
    gt_angle_rad = gt_angle_deg * np.pi / 180.0
    source = gt_distance * np.array([np.cos(gt_angle_rad), np.sin(gt_angle_rad)])
    sources.append(source)
    plt.scatter(*source)
    plt.axis('equal')

buffer_f_list, buffer_f_multimic = get_buffers()

In [ ]:
separate_doa(plot=True, polar=polar)

In [ ]:
multi_mic_doa(plot=True, polar=polar)

In [ ]:
kinetic_doa(plot=True, polar=polar)

# Moving noise source (like propellers)

# Full pipeline

In [ ]:
import pandas as pd

fname = 'results/new_square_test.pkl' # square array
df = pd.read_pickle(fname)
freq_i = 0
gt_angle_deg = 90

label_list = ['spectrum_dynamic', 'spectrum_delayed', 'spectrum_multimic']
n_angles = df.iloc[0].spectrum_multimic.shape[1] # n_freq x n_angles
angles = np.linspace(0, 360, n_angles)

n_degree_noise = len(df.degree_noise.unique())
n_it = len(df.it.unique())
boxplot_mat = np.empty((len(label_list), n_it, n_degree_noise)) # each slice will be one boxplot

for d, (degree_noise, df_degree_noise) in enumerate(df.groupby('degree_noise')):
    fig, axs = plt.subplots(1, len(label_list), sharey=True, sharex=True)
    fig.set_size_inches(10, 3)
    for counter, (__, row) in enumerate(df_degree_noise.iterrows()):
        for l, label in enumerate(label_list):
            if counter <= 3:
                axs[l].semilogy(angles, row[label][freq_i])
                
            unwrap_angles = np.array([angles[np.argmax(row[label][freq_i])], gt_angle_deg]) / 180 * np.pi
            unwrap_angles = np.unwrap(unwrap_angles)
            error = abs(unwrap_angles[1] - unwrap_angles[0])*180/np.pi
            
            boxplot_mat[l, counter, d] = error
        
    for label, ax in zip(label_list, axs):
        ax.axvline(gt_angle_deg, color='black', ls=':')
        ax.set_title(f'degree noise {degree_noise}')
        ax.set_xlabel('angle [deg]')
        ax.set_ylabel('spectrum [-]')
        ax.set_title(label)

In [ ]:
fig_box, ax_box = plt.subplots(1, len(label_list), sharex=True, sharey=True)
fig_box.set_size_inches(10, 5)
for l, label in enumerate(label_list):
    ax_box[l].boxplot(boxplot_mat[l], positions=df.degree_noise.unique(), widths=1.0)
    ax_box[l].set_title(label)